# GrIS AMOC interaction: real data

In [1]:
import numpy as np
import sys
from pathlib import Path

from scipy.ndimage import gaussian_filter

from tigramite import data_processing as pp

import pandas as pd
import xarray as xr

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import re
import pickle

from tigramite.causal_effects import CausalEffects

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 

In [2]:
repo_root = Path().resolve().parent
sys.path.append(str(repo_root))

In [3]:
import functions.sliding_windows as sw
import functions.indicators_computation as ind
import functions.significance_testing as sgtest

# helpers

In [4]:
def bootstrap_causal_autodependency(causal_effects, data, var_names, time_windows, n_points, estimator, n_bootstrap, boot_blocklength, detrend=False, seed=0):
    estimated_causal_effects = []
    intervention_data_windows = []
    data_copy=data.copy()
    data_length = data_copy.shape[0]

    # Initialize the bootstrap preds list: one list per bootstrap sample
    bootstrap_preds_by_sample = [[] for _ in range(n_bootstrap)]

    for window in time_windows:
        if window[-1] >= data_length:
            break  # Stop if the entire window is not included in the data

        # work on a temporary local copy for this window
        window_slice = data_copy[window, :].copy()
        if detrend:
            wlen = window_slice.shape[0]
            t = np.arange(wlen)
            for i in range(window_slice.shape[1]):
                xw_i = window_slice[:, i].astype(float).copy()
                xw_i = xw_i - xw_i.mean()
                slope, intercept = np.polyfit(t, xw_i, 1)
                detrended = xw_i - (slope * t + intercept)
                window_slice[:, i] = detrended


        dataframe = pp.DataFrame(window_slice, var_names=var_names)
        intervention_data_window, intervention_data_window_here = ind.produce_intervention_data(window_slice[:, -1], 0.01, n_points)
        
        # Fit causal effect model from observational data
        causal_effects.fit_total_effect(
                dataframe = dataframe, 
                estimator=estimator,
                adjustment_set='optimal',
                conditional_estimator=None,  
                data_transform=StandardScaler(),
                mask_type=None,
            )
        
        causal_effects.fit_bootstrap_of(
            method='fit_total_effect',
            method_args={'dataframe':dataframe, 'estimator':estimator, 'adjustment_set':'optimal', 'conditional_estimator':None, 'data_transform':StandardScaler(), 'mask_type':None,},
            boot_samples=n_bootstrap,
            boot_blocklength=boot_blocklength,
            seed=seed,
        )

        # Predict causal effect for the current window
        # and bootstrap samples
        window_estimated_causal_effects = causal_effects.predict_total_effect( 
            intervention_data=intervention_data_window_here,
            transform_interventions_and_prediction=True,)
        
        preds, _ = causal_effects.predict_bootstrap_of(
            method='predict_total_effect',
            method_args={'intervention_data':intervention_data_window_here, 'transform_interventions_and_prediction':True,},
            return_individual_bootstrap_results=True
        )
    
        # Append each window's predictions to the correct bootstrap sample
        for i in range(n_bootstrap):
            bootstrap_preds_by_sample[i].append(preds[i, :].tolist())      

        estimated_causal_effects.append(window_estimated_causal_effects)
        intervention_data_windows.append(intervention_data_window)


    return estimated_causal_effects, bootstrap_preds_by_sample, intervention_data_windows

In [5]:
def pipeline_causalEE_weighted_bootstrap(causal_effects, data, estimator, n_bootstrap, boot_blocklength, n_points, time_windows, var_names, edge_points_proportion=0.05, CI=90, detrend=False, seed=0):
    bootstrap_estimates = []

    causal_autodependencies, bootstrap_causal_estimates, intervention_data_windows = bootstrap_causal_autodependency(
        causal_effects, data, var_names=var_names, 
        time_windows=time_windows, n_points=n_points, 
        estimator=estimator, n_bootstrap=n_bootstrap, 
        boot_blocklength=boot_blocklength,
        detrend=detrend,
        seed=seed
    )

    densities = ind.windows_density(data, time_windows=time_windows)

    nb_edge_points=int(edge_points_proportion*n_points)
    effect_slopes, residuals, std_devs = ind.compute_effect_slopes_weighted(causal_autodependencies, densities, nb_edge_points, intervention_data_windows)

    for estimate in bootstrap_causal_estimates: # estimate = causal effects for one bootstrap sample
        slope, _, _ = ind.compute_effect_slopes_weighted(estimate, densities, nb_edge_points, intervention_data_windows)
        bootstrap_estimates.append(slope)

    if CI==90:
        confidence_interval = np.percentile(bootstrap_estimates, [5, 95], axis=0)
    elif CI==95:
        confidence_interval = np.percentile(bootstrap_estimates, [2.5, 97.5], axis=0)

    return effect_slopes, residuals, std_devs, bootstrap_estimates, confidence_interval

In [6]:
def boxplot_results_for_each_graph(graphs, nb_surrogates):
    """
    graphs: list of dicts, each dict must have:
      - 'label': str, name for x-axis and legend
      - 'values': array-like (time series for central slope)
      - 'bootstrap_values': list of arrays (bootstrap time series)
      - optional 'color': string (plot color)
    nb_surrogates: int, number of surrogates for p-value computation
    """

    def compute_slopes_and_pvalues(bootstrap_series_list):
        slopes, pvals = [], []
        for series in bootstrap_series_list:
            slope, _ = np.polyfit(np.arange(len(series)), series, 1)
            pval, _ = sgtest.p_value_from_fourier(series, nb_surrogates, slope)
            pvals.append(float(np.array(pval).flatten()[0]))
            slopes.append(slope)
        return slopes, pvals

    def extract_pval(value):
        slope, _ = np.polyfit(np.arange(len(value)), value, 1)
        pval, _ = sgtest.p_value_from_fourier(value, nb_surrogates, slope)
        return slope, float(np.array(pval).flatten()[0])

    fig = go.Figure()

    default_colors = ['rgb(0,153,198)', 'rgb(115,175,72)', 'rgb(255,127,14)', 'rgb(31,119,180)', 'rgb(44,160,44)']
    n_colors = len(default_colors)

    central_vals = []
    central_pvals = []
    x_labels = []
    all_bootstrap_slopes = []
    all_bootstrap_pvals = []
    colors = []

    # Compute slopes and pvals for each graph dynamically
    for i, graph in enumerate(graphs):
        label = graph['label']
        values = graph['values']
        bootstrap_values = graph['bootstrap_values']
        color = graph.get('color', default_colors[i % n_colors])

        slopes, pvals = compute_slopes_and_pvalues(bootstrap_values)
        slope, pval = extract_pval(values)

        central_vals.append(slope)
        central_pvals.append(pval)
        x_labels.append(label)
        all_bootstrap_slopes.append(slopes)
        all_bootstrap_pvals.append(pvals)
        colors.append(color)

    # Add boxplots (background)
    for i, slopes in enumerate(all_bootstrap_slopes):
        fig.add_trace(go.Box(
            y=slopes,
            x=[i] * len(slopes),
            name='',
            marker_color=colors[i],
            boxpoints=False,
            line_width=1,
            fillcolor=colors[i].replace('rgb', 'rgba').replace(')', ',0.1)'),
            showlegend=False
        ))

    # Add bootstrap points (colored, no legend)
    for i, (slopes, pvals, color) in enumerate(zip(all_bootstrap_slopes, all_bootstrap_pvals, colors)):
        for slope, pval in zip(slopes, pvals):
            symbol = 'circle' if pval < 0.01 else 'x'
            fig.add_trace(go.Scatter(
                x=[i],
                y=[slope],
                mode='markers',
                marker=dict(
                    symbol=symbol,
                    size=8,
                    color=color,
                    opacity=0.7
                ),
                hoverinfo='skip',
                showlegend=False,
            ))

    # Add central points (larger, black outline)
    for i, (val, pval, color) in enumerate(zip(central_vals, central_pvals, colors)):
        symbol = 'circle' if pval < 0.01 else 'x'
        fig.add_trace(go.Scatter(
            x=[i],
            y=[val],
            mode='markers+text',
            marker=dict(
                symbol=symbol,
                size=12,
                color=color,
                line=dict(width=2, color='black')
            ),
            showlegend=False
        ))

    # Add legend markers (black only, once)
    fig.add_trace(go.Scatter(
        x=[None], y=[None], mode='markers',
        marker=dict(symbol='circle', size=9, color='black'),
        name='p < 0.01'
    ))
    fig.add_trace(go.Scatter(
        x=[None], y=[None], mode='markers',
        marker=dict(symbol='x', size=9, color='black'),
        name='p ≥ 0.01'
    ))

    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(x_labels))),
            ticktext=x_labels
        ),
        yaxis_title="Slope",
        height=600,
        width=750 + 60 * (len(graphs) - 3),  # widen plot if many graphs
        template='simple_white',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='center',
            x=0.5
        )
    )

    fig.show()

# 1. Data import and visualisation

## Melt rates and AMOC Caesar

In [7]:
# GIS, Arctic summer temperature, AMOC
file_path = "../data/JJA_CWG_NU.xlsx"
JJA_CWG_NU_AMOC = pd.read_excel(file_path)

JJA_CWG_NU_AMOC = JJA_CWG_NU_AMOC[["Year", "JJA", "CWG", "NU", "AMOC_Caesar"]]

In [8]:
JJA_CWG_NU_AMOC

,Year,JJA,CWG,NU,AMOC_Caesar
0,2013,6.533333,0.034009,2.164248,-8.759240
1,2012,8.033333,11.038916,1.087561,-8.863189
2,2011,7.500000,1.992249,3.452864,-8.610168
3,2010,8.100000,1.739967,2.436886,-8.898290
4,2009,7.566667,1.838914,1.835124,-8.817147
...,...,...,...,...,...
359,1654,NaN,NaN,0.145832,NaN
360,1653,NaN,NaN,0.793482,NaN
361,1652,NaN,NaN,-0.847182,NaN
362,1651,NaN,NaN,-1.088726,NaN


In [9]:
df_1871_2013 = JJA_CWG_NU_AMOC[JJA_CWG_NU_AMOC['Year'] >= 1871]
df_1871_2013 = df_1871_2013.iloc[::-1].reset_index(drop=True)
years_1871_2013 = list(df_1871_2013["Year"])
df_1871_2013 = df_1871_2013.set_index('Year')

In [10]:
sigma = 30

AMOC_Caesar = df_1871_2013["AMOC_Caesar"].values
AMOC_Caesar_det = AMOC_Caesar - gaussian_filter(AMOC_Caesar, sigma=sigma)

CWG = df_1871_2013["CWG"].values
CWG_det = CWG - gaussian_filter(CWG, sigma=sigma)
CWG_log = np.log(CWG + np.abs(CWG.min()) + 1)
CWG_log_det = CWG_log - gaussian_filter(CWG_log, sigma=sigma)

### AMOC

In [11]:
years_1871_2013 = np.arange(1871, 2014)  # 1871 - 2013
years_1871_2016 = np.arange(1871, 2017)  # 1871 - 2016

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=years_1871_2016, y=AMOC_Caesar, mode='lines', name='Caesar index (°C)', line=dict(color='#00AAD4')))
fig.update_layout(plot_bgcolor='rgba(215, 238, 244, 0.3)',width=600,height=400, xaxis_title='Year', yaxis_title='Caesar index (°C)',
                  legend=dict(x=0.01, y=0.03))

In [13]:
#fig.write_image('fig_AMOC_Caesar.svg')

In [14]:
window_length=70
window_step=1
time_windows, center_points = sw.get_centered_sliding_windows(window_length, window_step, t_len=len(years_1871_2013))

In [15]:
AC_Caesar = ind.regular_autocorrelation(AMOC_Caesar_det, time_windows, lag=1, detrend=True)

slopeAC_Caesar, interceptAC_Caesar = np.polyfit(np.arange(len(AC_Caesar)), AC_Caesar, 1)
pvalAC_Caesar, _ = sgtest.p_value_from_fourier(AC_Caesar, 10000, slopeAC_Caesar)

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=years_1871_2013+window_length//2, y=AC_Caesar, mode='lines', name='AC1 Caesar index', line=dict(color='#00AAD4')))
fig.add_trace(go.Scatter(x=years_1871_2013+window_length//2, y=slopeAC_Caesar * np.arange(len(AC_Caesar)) + interceptAC_Caesar, mode='lines', name=f'p: {pvalAC_Caesar:.2f}', line=dict(color='#00AAD4', dash='dash')))

fig.update_layout(xaxis_range = [1871, 2016], width=600,height=400, xaxis_title='Year', yaxis_title='AC1 AMOC index',
                  legend=dict(x=0.01, y=0.99), template='simple_white')

## CWG melt rates

In [17]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=list(range(1871, 2013)),y=CWG_log,mode='lines',name='CWG (log)',line=dict(color='#2CA02C')),secondary_y=False)
fig.add_trace(go.Scatter(x=list(range(1871, 2013)),y=CWG,mode='lines',name='CWG (z-scores)',line=dict(color='#5FD35F')),secondary_y=True)

fig.update_layout(plot_bgcolor='rgba(215, 238, 244, 0.3)',width=630,height=400,xaxis_title='Year',legend=dict(x=0.01, y=0.98))
fig.update_yaxes(title_text='CWG melt rates (log)', secondary_y=False)
fig.update_yaxes(title_text='CWG melt rates (z-scores)', secondary_y=True)

fig.show()

In [18]:
#fig.write_image('fig_CWG.svg')

In [19]:
window_length=70
window_step=1
time_windows, center_points = sw.get_centered_sliding_windows(window_length, window_step, t_len=len(years_1871_2013))

In [20]:
AC_CWG_log = ind.regular_autocorrelation(CWG_log_det, time_windows, lag=1, detrend=True)
slopeAC_CWG, interceptAC_CWG = np.polyfit(np.arange(len(AC_CWG_log)), AC_CWG_log, 1)
pvalAC_CWG, _ = sgtest.p_value_from_fourier(AC_CWG_log, 10000, slopeAC_CWG)

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=years_1871_2013+window_length//2, y=AC_CWG_log, mode='lines', name='AC1 CWG', line=dict(color='#00AAD4')))
fig.add_trace(go.Scatter(x=years_1871_2013+window_length//2, y=slopeAC_CWG * np.arange(len(AC_CWG_log)) + interceptAC_CWG, mode='lines', name=f'p: {pvalAC_CWG:.2f}', line=dict(color='#00AAD4', dash='dash')))
    
fig.update_layout(xaxis_range = [1871, 2016], width=600,height=400, xaxis_title='Year', yaxis_title='AC1 GrIS melt rates',
                  legend=dict(x=0.01, y=0.99), template='simple_white')

## Salinity

In [22]:
# --- Global parameters ---
years_1900_2019 = np.arange(1900, 2020)  # data years
months = ['%02d' % m for m in range(1, 13)]
tlen = len(years_1900_2019) * 12         # total length in months
dth = 300                      # maximum depth in meters

In [23]:
SNN1 = np.load('../data/EN422_NA_salinity_am_d300.npy')

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=years_1900_2019, y=-SNN1, mode='lines', name='SNN1 (44–66°N, 289–330°E)', line=dict(color='#00AAD4')))
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Salinity Anomaly (psu)',
    height=400, width=600,
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor='rgba(215, 238, 244, 0.3)'
)
fig.show()

In [25]:
#fig.write_image('fig_salinity_annual.svg')

In [26]:
window_length=70
window_step=1

time_windows, center_points = sw.get_centered_sliding_windows(window_length, window_step, t_len=len(years_1900_2019))

In [27]:
AC_SNN1 = ind.regular_autocorrelation(SNN1, time_windows, lag=1, detrend=True)

In [28]:
slopeAC_SNN1, interceptAC_SNN1 = np.polyfit(np.arange(len(AC_SNN1)), AC_SNN1, 1)
pvalAC_SNN1, _ = sgtest.p_value_from_fourier(AC_SNN1, 10000, slopeAC_SNN1)

In [29]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=years_1900_2019+window_length//2, y=AC_SNN1, mode='lines', name='AC SNN1', line=dict(color='#2CA02C')))
fig.add_trace(go.Scatter(x=years_1900_2019+window_length//2, y=slopeAC_SNN1 * np.arange(len(AC_SNN1)) + interceptAC_SNN1, mode='lines', name=f'pv: {pvalAC_SNN1:.2f}', line=dict(color='#2CA02C', dash='dash')))

fig.update_layout(xaxis_range=[1900, 2020], xaxis_title='Years', yaxis_title='AC1 salinities', height=400, width=600, template='simple_white', legend=dict(x=0.02, y=0.99))
fig.show()

## Freshwater

In [30]:
# Freshwater
ds = xr.open_dataset("../data/FWF17.v3_b.nc")

In [31]:
# Create a mask for the North Atlantic Ocean
north_atlantic_id = 76
north_atlantic_mask = ds['ocean_basins'] == north_atlantic_id

In [32]:
# Apply the mask to select data only for the North Atlantic Ocean
gr_tundra_runoff_na = ds['runoff_tundra'].where(north_atlantic_mask, 0).where(ds['LSMGr'] == 1, 0)
gr_runoff_ice_na = ds['runoff_ice'].where(north_atlantic_mask, 0).where(ds['LSMGr'] == 1, 0)
gr_solid_ice_na = ds['solid_ice'].where(north_atlantic_mask, 0).where(ds['LSMGr'] == 1, 0) 

tundra_runoff_na = ds['runoff_tundra'].where(north_atlantic_mask, 0)
runoff_ice_na = ds['runoff_ice'].where(north_atlantic_mask, 0)
solid_ice_na = ds['solid_ice'].where(north_atlantic_mask, 0)

In [33]:
# Sum over the spatial dimensions (X, Y) for each month, then resample by year to sum monthly values
gr_runoff_tundra_yearly = gr_tundra_runoff_na.sum(dim=["X", "Y"]).resample(TIME="YE").sum()
gr_runoff_ice_yearly = gr_runoff_ice_na.sum(dim=["X", "Y"]).resample(TIME="YE").sum()
gr_solid_ice_yearly = gr_solid_ice_na.sum(dim=["X", "Y"]).resample(TIME="YE").sum()

runoff_tundra_yearly = tundra_runoff_na.sum(dim=["X", "Y"]).resample(TIME="YE").sum()
runoff_ice_yearly = runoff_ice_na.sum(dim=["X", "Y"]).resample(TIME="YE").sum()
solid_ice_yearly = solid_ice_na.sum(dim=["X", "Y"]).resample(TIME="YE").sum()

In [34]:
gr_total_freshwater = gr_runoff_tundra_yearly + gr_runoff_ice_yearly + gr_solid_ice_yearly
years_1958_2016 = gr_total_freshwater['TIME']
total_freshwater = runoff_tundra_yearly + runoff_ice_yearly + solid_ice_yearly

fig = go.Figure()
fig.add_trace(go.Scatter(x=years_1958_2016, y=total_freshwater, mode='lines', name='Total FWF', line=dict(color='#2CA02C')))
fig.add_trace(go.Scatter(x=years_1958_2016, y=gr_total_freshwater, mode='lines', name='GrIS FWF', line=dict(color='#5FD35F')))
fig.update_layout(plot_bgcolor='rgba(215, 238, 244, 0.3)', width=600,height=400, xaxis_title='Year', yaxis_title='FWF into the NA (km3)', legend=dict(x=0.01, y=0.98))

In [35]:
#fig.write_image('fig_FWF_total_GrIS.svg')
#fig.write_image('fig_FWF_total.svg')

In [36]:
years_1958_2016 = np.array([pd.Timestamp(year).year for year in years_1958_2016.values])

In [37]:
window_length=30
window_step=1
time_windows, center_points = sw.get_centered_sliding_windows(window_length, window_step, t_len=len(years_1958_2016))

In [38]:
total_freshwater_det = total_freshwater - gaussian_filter(total_freshwater, sigma=30)
gr_total_freshwater_det = gr_total_freshwater - gaussian_filter(gr_total_freshwater, sigma=30)

In [39]:
AC_totalFWF = ind.regular_autocorrelation(total_freshwater_det, time_windows, lag=1, detrend=True)
slopeAC_FWF, interceptAC_FWF = np.polyfit(np.arange(len(AC_totalFWF)), AC_totalFWF, 1)
pvalAC_FWF, _ = sgtest.p_value_from_fourier(AC_totalFWF, 10000, slopeAC_FWF)

In [40]:
AC_grFWF = ind.regular_autocorrelation(gr_total_freshwater_det, time_windows, lag=1, detrend=True)
slopeAC_grFWF, interceptAC_grFWF = np.polyfit(np.arange(len(AC_grFWF)), AC_grFWF, 1)
pvalAC_grFWF, _ = sgtest.p_value_from_fourier(AC_grFWF, 10000, slopeAC_grFWF)

In [41]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=years_1958_2016+window_length//2, y=AC_totalFWF, mode='lines', name='AC1 Total FWF', line=dict(color='#2CA02C')))
fig.add_trace(go.Scatter(x=years_1958_2016+window_length//2, y=slopeAC_FWF * np.arange(len(AC_totalFWF)) + interceptAC_FWF, mode='lines', name=f'p: {pvalAC_FWF:.2f}', line=dict(color='#2CA02C', dash='dash')))

fig.add_trace(go.Scatter(x=years_1958_2016+window_length//2, y=AC_grFWF, mode='lines', name='AC1 GrIS FWF', line=dict(color='#5FD35F')))
fig.add_trace(go.Scatter(x=years_1958_2016+window_length//2, y=slopeAC_grFWF * np.arange(len(AC_grFWF)) + interceptAC_grFWF, mode='lines', name=f'p: {pvalAC_grFWF:.2f}', line=dict(color='#5FD35F', dash='dash')))

fig.update_layout(xaxis_range = [1958, 2017], width=600,height=400, xaxis_title='Year', yaxis_title='AC1 FWF into the NA', legend=dict(x=0.01, y=0.99), template='simple_white')

In [42]:
# fig.write_image('fig_AC1_FWF.svg')

In [43]:
wl_list=[20, 30, 40]
window_step=1
nb_surrogates=10000

fig= go.Figure()
for wl in wl_list:
    time_windows, center_points = sw.get_centered_sliding_windows(wl, window_step, t_len=len(years_1958_2016))
    AC_totalFWF_wl = ind.regular_autocorrelation(total_freshwater_det, time_windows, lag=1, detrend=True)
    slopeAC_FWF_wl, interceptAC_FWF_wl = np.polyfit(np.arange(len(AC_totalFWF_wl)), AC_totalFWF_wl, 1)
    pvalAC_FWF_wl, _ = sgtest.p_value_from_fourier(AC_totalFWF_wl, nb_surrogates, slopeAC_FWF_wl)

    fig.add_trace(go.Scatter(x=years_1958_2016+wl//2, y=AC_totalFWF_wl, mode='lines', name=f'wl={wl}', line=dict(color='#2CA02C', dash='dashdot' if wl==20 else 'solid' if wl==30 else 'dot')))
    fig.add_trace(go.Scatter(x=years_1958_2016+wl//2, y=slopeAC_FWF_wl * np.arange(len(AC_totalFWF_wl)) + interceptAC_FWF_wl, mode='lines', name=f'p: {pvalAC_FWF_wl:.2f}', line=dict(color="#2CA02C", dash='dashdot' if wl==20 else 'solid' if wl==30 else 'dot'), opacity=0.5))


fig.update_layout(xaxis_range = [1958, 2017], width=600,height=400, xaxis_title='Year', yaxis_title='AC1 total FWF into the NA', legend=dict(x=0.01, y=1), template='simple_white')

In [44]:
# fig.write_image('fig_AC1_FWF_window.svg')

# 2. Causal analysis

## AMOC only

In [45]:
n_bootstrap = 100
boot_blocklength=4
nb_surrogates = 10000

In [46]:
t_len = len(years_1871_2013)

window_length = 70
window_step = 1

time_windows, center_points = sw.get_centered_sliding_windows(window_length, window_step, t_len)

In [47]:
n_points = 150

In [48]:
# assuming lag 1 influence of AMOC on AMOC
graph =  np.array([[['', '-->']],], dtype='<U3')

X = [(0,-1)]
Y = [(0,0)]

causal_effects = CausalEffects(graph, graph_type='stationary_dag', X=X, Y=Y, S=None, 
                            hidden_variables=None, verbosity=0)

In [49]:
# causal_LinReg_AMOC, _, _, bootstrap_LinReg_AMOC, CI_LinReg_AMOC = pipeline_causalEE_weighted_bootstrap(
#     causal_effects, AMOC_Caesar_det.reshape(-1, 1), LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows, var_names=[r'$AMOC$'], edge_points_proportion=0.15, CI=90, detrend=True)

## GrIS and AMOC

In [50]:
def compute_and_save_plot_data(graphs, nb_surrogates, save_path='boxplot.pkl'):
    def compute_slopes_and_pvalues(bootstrap_series_list):
        slopes, pvals = [], []
        for series in bootstrap_series_list:
            slope, _ = np.polyfit(np.arange(len(series)), series, 1)
            pval, _ = sgtest.p_value_from_fourier(series, nb_surrogates, slope, seed=0)
            pvals.append(float(np.array(pval).flatten()[0]))
            slopes.append(slope)
        return slopes, pvals

    def extract_pval(value):
        slope, _ = np.polyfit(np.arange(len(value)), value, 1)
        pval, _ = sgtest.p_value_from_fourier(value, nb_surrogates, slope, seed=0)
        return slope, float(np.array(pval).flatten()[0])

    processed_graphs = []

    for i, graph in enumerate(graphs):
        slopes, pvals = compute_slopes_and_pvalues(graph['bootstrap_values'])
        central_slope, central_pval = extract_pval(graph['values'])

        processed_graph = {
            'label': graph['label'],
            'central_slope': central_slope,
            'central_pval': central_pval,
            'bootstrap_slopes': slopes,
            'bootstrap_pvals': pvals,
            'color': graph.get('color', f"rgb({i*40%255},{i*60%255},{i*80%255})")
        }
        processed_graphs.append(processed_graph)

    with open(save_path, 'wb') as f:
        pickle.dump(processed_graphs, f)

In [51]:
def plot_precomputed_results_boxplot(processed_graphs):

    fig = go.Figure()

    central_vals = [g['central_slope'] for g in processed_graphs]
    central_pvals = [g['central_pval'] for g in processed_graphs]
    x_labels = [g['label'] for g in processed_graphs]
    all_bootstrap_slopes = [g['bootstrap_slopes'] for g in processed_graphs]
    all_bootstrap_pvals = [g['bootstrap_pvals'] for g in processed_graphs]
    colors = [g['color'] for g in processed_graphs]

    for i, slopes in enumerate(all_bootstrap_slopes):
        q5, q95 = np.percentile(slopes, [5, 95])
        median_or_mean = np.mean(slopes)
        color = colors[i]

        fig.add_trace(go.Box(
            x=[i] * len(slopes),
            q1=[q5],
            median=[median_or_mean],
            q3=[q95],
            lowerfence=[q5],
            upperfence=[q95],
            boxpoints=False,
            marker_color=color,
            line_width=1,
            fillcolor=color.replace('rgb', 'rgba').replace(')', ',0.1)'),
            name='',
            showlegend=False
        ))

    for i, (slopes, pvals, color) in enumerate(zip(all_bootstrap_slopes, all_bootstrap_pvals, colors)):
        for slope, pval in zip(slopes, pvals):
            symbol = 'circle' if pval < 0.05 else 'x'
            fig.add_trace(go.Scatter(
                x=[i],
                y=[slope],
                mode='markers',
                marker=dict(symbol=symbol, size=7, color=color, opacity=0.6),
                hoverinfo='skip',
                showlegend=False,
            ))

    for i, (val, pval, color) in enumerate(zip(central_vals, central_pvals, colors)):
        if pval < 0.01:
            fig.add_trace(go.Scatter(
                x=[i], y=[val], mode='markers',
                marker=dict(symbol='circle', size=12, color=color, line=dict(width=2, color='black')),
                showlegend=False
            ))
        elif 0.01 <= pval < 0.05:
            fig.add_trace(go.Scatter(
                x=[i], y=[val], mode='markers',
                marker=dict(symbol='circle', size=12, color=color, line=dict(width=2, color='black')),
                showlegend=False
            ))
            fig.add_trace(go.Scatter(
                x=[i], y=[val], mode='markers',
                marker=dict(symbol='x', size=6, color='red'),
                showlegend=False
            ))
        else:
            fig.add_trace(go.Scatter(
                x=[i], y=[val], mode='markers',
                marker=dict(symbol='x', size=12, color=color, line=dict(width=2, color='black')),
                showlegend=False
            ))

    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                             marker=dict(symbol='circle', size=9, color='black'),
                             name='p < 0.05'))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                             marker=dict(symbol='x', size=9, color='black'),
                             name='p ≥ 0.05'))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                             marker=dict(symbol='x', size=9, color='red'),
                             name='p ≥ 0.01'))
    x_labels = ["Graph A"] + [f"Graph(τ={i})" for i in range(6)]
    fig.update_layout(
        title='Central values and bootstrap slopes with 90% confidence intervals',
        xaxis=dict(tickmode='array', tickvals=list(range(len(x_labels))), ticktext=x_labels),
        yaxis=dict(title='Slope (×10³) (yr⁻¹)', #tickvals=[-0.003, -0.002,-0.001, 0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019],
                   #ticktext=['-3', '-2', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19']),
                   #tickvals=[-0.02, -0.015, -0.01, -0.002, 0, 0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04],
                   #ticktext=['-20', '-15', '-10', '-2', '0', '2', '4', '6', '8', '10', '12', '15', '20', '25', '30', '35', '40']),
                #    tickvals = [-0.005, 0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03],
                #      ticktext = ['-5', '0', '5', '10', '15', '20', '25', '30']),
                     tickvals = [-0.01, -0.005, 0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04],
                     ticktext = ['-10', '-5', '0', '5', '10', '15', '20', '25', '30', '35', '40']),
        height=600,
        width=550 + 60 * (len(processed_graphs) - 3),
        template='simple_white',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5)
    )

    # Ligne horizontale au niveau de la première pente centrale
    fig.add_trace(go.Scatter(
        x=[0, len(processed_graphs) - 1],
        y=[central_vals[0], central_vals[0]],
        mode='lines',
        line=dict(color='rgb(0,153,198)', dash='dot'),
        opacity=0.6,
        showlegend=False
    ))

    return fig

In [52]:
def compute_CE_for_all_lags(dataAMOC_det, dataGrIS_det, graphs, graphAMOC, time_windows, n_bootstrap=100, boot_blocklength=4, CI=90, detrend=False):
    results = {}

    # --- Graph A: AMOC only ---
    print("Analyzing Graph A (AMOC only)")
    causal_effects_AMOC = CausalEffects(
        graphAMOC,
        graph_type='stationary_dag',
        X=[(0, -1)],
        Y=[(0, 0)],
        S=None,
        hidden_variables=None,
        verbosity=0
    )

    data_AMOC_only = dataAMOC_det.reshape(-1, 1)
    var_names_AMOC = [r'$AMOC$']

    causal_AMOC, _, _, bootstrap_AMOC, _ = pipeline_causalEE_weighted_bootstrap(
        causal_effects_AMOC,
        data_AMOC_only,
        LinearRegression(),
        n_bootstrap=n_bootstrap,
        boot_blocklength=boot_blocklength,
        data_transform=StandardScaler(),
        n_points=n_points,
        time_windows=time_windows,
        var_names=var_names_AMOC,
        edge_points_proportion=edge_points_proportion,
        CI=CI,
        detrend=detrend
    )

    results["Graph A"] = (causal_AMOC, bootstrap_AMOC)

    # --- Graphs for GrIS → AMOC ---
    for i, graph in enumerate(graphs):
        print(f"Analyzing Graph {i} (GrIS → AMOC)")
        causal_effects = CausalEffects(
            graph,
            graph_type='stationary_dag',
            X=[(1, -1)],
            Y=[(1, 0)],
            S=None,
            hidden_variables=None,
            verbosity=0
        )

        data_pair = np.hstack([
            dataGrIS_det.reshape(-1, 1),
            dataAMOC_det.reshape(-1, 1)
        ])
        var_names = [r'$GrIS$', r'$AMOC$']

        causal_AMOC_GrIS, _, _, bootstrap_AMOC_GrIS, _ = pipeline_causalEE_weighted_bootstrap(
            causal_effects,
            data_pair,
            LinearRegression(),
            n_bootstrap=n_bootstrap,
            boot_blocklength=boot_blocklength,
            data_transform=StandardScaler(),
            n_points=n_points,
            time_windows=time_windows,
            var_names=var_names,
            edge_points_proportion=edge_points_proportion,
            CI=CI,
            detrend=detrend
        )

        results[f"Graph {i}"] = (causal_AMOC_GrIS, bootstrap_AMOC_GrIS)

    return results

In [53]:
def create_graph_from_results(results):
    graphs = []
    for i, (label, (values, bootstrap_values)) in enumerate(results.items()):
        if 'A' in label or 'AMOC' in label:
            color = 'rgb(0,153,198)'  # AMOC blue
        else:
            color = 'rgb(115,175,72)'  # GrIS green

        graphs.append({
            'label': label,
            'values': values,
            'bootstrap_values': bootstrap_values,
            'color': color
        })
    return graphs

In [54]:
graphAMOC = np.array([[['', '-->', '', '', '', '']],], dtype='<U3')

graphsCE = [
    np.array([[['', '-->', '', '', '', ''], ['-->', '', '', '', '', '']],
            [['<--', '', '', '', '', ''], ['', '-->', '', '', '', '']],], dtype='<U3'), 

    np.array([[['', '-->', '', '', '', ''], ['', '-->', '', '', '', '']],
                [['', '', '', '', '', ''], ['', '-->', '', '', '', '']],], dtype='<U3'),

    np.array([[['', '-->', '', '', '', ''], ['', '', '-->', '', '', '']],
                [['', '', '', '', '', ''], ['', '-->', '', '', '', '']],], dtype='<U3'),

    np.array([[['', '-->', '', '', '', ''], ['', '', '', '-->', '', '']],
                [['', '', '', '', '', ''], ['', '-->', '', '', '', '']]], dtype='<U3'),

    np.array([[['', '-->', '', '', '', ''], ['', '', '', '', '-->', '']],
                [['', '', '', '', '', ''], ['', '-->', '', '', '', '']]], dtype='<U3'),

    np.array([[['', '-->', '', '', '', ''], ['', '', '', '', '', '-->']],
                [['', '', '', '', '', ''], ['', '-->', '', '', '', '']]], dtype='<U3'), 
    ]

In [55]:
graphAMOC = np.array([[['', '-->', '', '', '', '']],], dtype='<U3')

In [56]:
def plot_custom_results(AC_AMOC, AC_GIS,
                        causal_AMOC, causal_GIS_AMOC,
                        CI_AMOC, CI_GIS_AMOC,
                        time, window_length, lag=1, GrIS_proxy='CWG', nb_surrogates=10000,
                        include_CI_first_row=False):
    """
    include_CI_first_row: bool
        If True, plot CI fills for the first row traces that have CI provided.
        If False, plot only the lines (no CI fill).
    """

    start_index = window_length // 2
    end_index = len(time) - (window_length // 2)
    x = time[start_index:end_index]

    fig = make_subplots(
        rows=2, cols=1,
        vertical_spacing=0.1,
        row_heights=[0.5, 0.5],
        specs=[[{"secondary_y": True}], [{"secondary_y": False}]],
    )

    def add_line_with_CI(fig, x, y, CI, color, name, row, col,
                         include_CI=True, alpha=0.2, secondary_y=False):
        """
        If include_CI is False -> only plot the y line.
        If include_CI is True and CI is not None -> plot filled CI and the line.
        CI is expected as (lower_bound_array, upper_bound_array) aligned with x.
        """
        # If CI exists and user wants to include it, draw the filled band
        if include_CI and CI is not None:
            lower_bound, upper_bound = CI
            # safe convert rgb->rgba (if color already rgba, keep as is)
            if color.startswith('rgb(') and not color.startswith('rgba('):
                fill_color = color.replace('rgb', 'rgba').replace(')', f', {alpha})')
            else:
                # fallback: append alpha if not present
                fill_color = color
            fig.add_trace(go.Scatter(
                x=np.concatenate([x, x[::-1]]),
                y=np.concatenate([upper_bound, lower_bound[::-1]]),
                fill='toself',
                fillcolor=fill_color,
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo="skip",
                showlegend=False
            ), row=row, col=col, secondary_y=secondary_y)

        # Always plot the main line
        fig.add_trace(go.Scatter(
            x=x, y=y, mode='lines',
            line=dict(color=color),
            name=name
        ), row=row, col=col, secondary_y=secondary_y)

    # --- Row 1: AC_AMOC, AC_GIS ---
    # --- AC_AMOC ---
    fig.add_trace(go.Scatter(x=x, y=AC_AMOC, mode='lines', name='AC1 AMOC',
                             line=dict(color='rgb(255,165,0)')), row=1, col=1)
    slope, intercept = np.polyfit(x, AC_AMOC, 1)
    pv, _ = sgtest.p_value_from_fourier(AC_AMOC, nb_surrogates, slope, seed=0)
    p_str = "p < 0.001" if pv < 0.001 else f"p = {pv:.3f}"
    fig.add_trace(go.Scatter(
        x=x, y=slope * x + intercept, mode='lines',
        name=f'{p_str}',
        line=dict(color='orange', dash="dot")
    ), row=1, col=1)

    # --- AC_GIS (grey, no CI) ---
    fig.add_trace(go.Scatter(x=x, y=AC_GIS, mode='lines', name=f'AC1 {GrIS_proxy}',
                             line=dict(color='grey')), row=1, col=1)
    slope, intercept = np.polyfit(x, AC_GIS, 1)
    pv, _ = sgtest.p_value_from_fourier(AC_GIS, nb_surrogates, slope, seed=0)
    p_str = "p < 0.001" if pv < 0.001 else f"p = {pv:.3f}"
    fig.add_trace(go.Scatter(
        x=x, y=slope * x + intercept, mode='lines',
        name=f'{p_str}',
        line=dict(color='grey', dash="dot")
    ), row=1, col=1)

    # --- Row 2: causal indicators ---
    for y, CI, color, name in [
        (causal_AMOC, CI_AMOC, 'rgb(0,170,212)', 'Graph A'),
        (causal_GIS_AMOC, CI_GIS_AMOC, 'rgb(115,175,72)', f'Graph(τ={lag})'),
    ]:
        # For row 2 we keep CI as before (always include if provided)
        add_line_with_CI(fig, x, y, CI, color, name, 2, 1, include_CI=True)
        slope, intercept = np.polyfit(x, y, 1)
        pv, _ = sgtest.p_value_from_fourier(y, nb_surrogates, slope, seed=0)
        p_str = "p < 0.001" if pv < 0.001 else f"p = {pv:.3f}"
        fig.add_trace(go.Scatter(
            x=x, y=slope * x + intercept, mode='lines',
            name=f'{p_str}',
            line=dict(color=color, dash="dot")
        ), row=2, col=1)

    fig.update_layout(
        template='plotly_white',
        width=900,
        height=650,
        xaxis1=dict(range=[time[0], time[-1]], title='Year'),
        xaxis2=dict(range=[time[0], time[-1]], title='Year'),
        yaxis=dict(range=[-0.2, 1], title='AC1', tickvals=[0, 0.5, 1]),
        yaxis2=dict(range=[-0.2, 1], tickvals=[0, 0.5, 1], title='Restoring rate'),
        yaxis3=dict(range=[-0.2, 1], tickvals=[0, 0.5, 1], title='DCE'),
    )

    return fig

In [57]:
def sensitivity_analysis(dataAMOC, dataGrIS=None, graphs=graphsCE, sigma_ls=[20, 30, 40], window_length_ls = [60, 70, 80], nb_surrogates=10000, detrend=True):
    results = {}  # Use a dictionary to store p-values with descriptive keys
    t_len = len(dataAMOC)

    for i, graph in enumerate(graphs):
        if dataGrIS is not None:
            print(f"Analyzing graph {i}")
            causal_effects = CausalEffects(graph, graph_type='stationary_dag', X=[(1, -1)], Y=[(1, 0)], S=None, hidden_variables=None, verbosity=0)
            var_names = [r'$GrIS$', r'$AMOC$']
        else:
            print(f"Analyzing graph with AMOC only")
            causal_effects = CausalEffects(graph, graph_type='stationary_dag', X=[(0, -1)], Y=[(0, 0)], S=None, hidden_variables=None, verbosity=0)
            var_names = [r'$AMOC$']

        for sigma in sigma_ls:
            AMOC_det = dataAMOC - gaussian_filter(dataAMOC, sigma=sigma)
            if dataGrIS is not None:
                GrIS_det = dataGrIS - gaussian_filter(dataGrIS, sigma=sigma)
                data = np.hstack([GrIS_det.reshape(-1, 1), AMOC_det.reshape(-1, 1)])
            else:
                data = AMOC_det.reshape(-1, 1)

            for window_length in window_length_ls:
                time_windows, _ = sw.get_centered_sliding_windows(window_length, 1, t_len)
                causal_effect, _, _ = ind.pipeline_causalEE_weighted(
                    causal_effects, data, LinearRegression(), data_transform=StandardScaler(),
                    n_points=n_points, time_windows=time_windows, var_names=var_names,
                    edge_points_proportion=edge_points_proportion,
                    detrend=detrend
                )
                
                slope, _ = np.polyfit(range(len(causal_effect)), causal_effect, 1)
                pv, _ = sgtest.p_value_from_fourier(causal_effect, nb_surrogates, slope, seed=0)

                key = f"pv{i}_{sigma}_{window_length}"
                results[key] = pv

    return results

In [58]:
def plot_pvalue_heatmaps_from_results(
    results,
    bandwidths,
    window_lengths,
    threshold=0.05,
    AMOC_only=False
):
    # Initialiser les p-values pour chaque graphe (même si on n'affichera qu'un seul)
    pv_lists_by_graph = {i: np.full((len(bandwidths), len(window_lengths)), np.nan) for i in range(6)}

    for label, pv in results.items():
        match = re.match(r'pv(\d+)_(\d+)_(\d+)', label)
        if match:
            graph_i = int(match.group(1))
            sigma = int(match.group(2))
            window_length = int(match.group(3))

            if graph_i in pv_lists_by_graph:
                try:
                    y_idx = bandwidths.index(sigma)
                    x_idx = window_lengths.index(window_length)
                    pv_lists_by_graph[graph_i][y_idx, x_idx] = pv
                except ValueError:
                    continue  # ignore if sigma or window_length not in lists

    # Choix de la disposition en fonction de AMOC_only
    if AMOC_only:
        rows, cols = 1, 1
        subplot_titles = [f"Graph A"]
    else:
        rows, cols = 2, 3
        subplot_titles = [f"Graph(τ={i})" for i in range(6)]

    fig = make_subplots(rows=rows, cols=cols,
                        subplot_titles=subplot_titles,
                        horizontal_spacing=0.1, vertical_spacing=0.2)

    def get_index(i):
        return i // 3 + 1, i % 3 + 1

    # détermine quels indices de graphes on va tracer
    if AMOC_only:
        graph_indices = [0]
    else:
        graph_indices = list(range(6))

    for idx_i, i in enumerate(graph_indices):
        # si AMOC_only, on mappe l'unique position (1,1)
        if AMOC_only:
            row, col = 1, 1
        else:
            row, col = get_index(i)
        Z = pv_lists_by_graph[i]

        fig.add_trace(go.Heatmap(
            z=Z,
            x=[str(wl) for wl in window_lengths],  # treat as categorical
            y=[str(bw) for bw in bandwidths],      # treat as categorical
            colorscale='YlOrBr',
            colorbar=dict(title='p-value') if (not AMOC_only and i == 2) or (AMOC_only) else None,
            zmin=0, zmax=0.3,
            showscale=(AMOC_only or (i == 2))
        ), row=row, col=col)


        # annotations (◯ si significatif, ✕ sinon)
        for y_idx, bw in enumerate(bandwidths):
            for x_idx, wl in enumerate(window_lengths):
                p = Z[y_idx, x_idx]
                if not np.isnan(p):
                    symbol = '◯' if p < threshold else '✕'
                    fig.add_trace(go.Scatter(
                        x=[wl], y=[bw],
                        text=[symbol],
                        mode='text',
                        textfont=dict(color='black', size=15),
                        showlegend=False,
                    ), row=row, col=col)

        # axes
        if row == rows:  # dernier rang -> label x
            fig.update_xaxes(title_text="Window length", row=row, col=col)
        if col == 1:  # première colonne -> label y
            fig.update_yaxes(title_text="Bandwidth", row=row, col=col)

    # Ajuster la taille de la figure en fonction du mode
    if AMOC_only:
        fig.update_layout(
            height=320, width=370,
            margin=dict(t=50),
        )
    else:
        fig.update_layout(
            height=600, width=900,
            margin=dict(t=50),
        )
    fig.update_xaxes(type='category')
    fig.update_yaxes(type='category')

    return fig

## CWG and Caesar

In [59]:
n_bootstrap = 100
boot_blocklength = 4
n_points = 150
edge_points_proportion = 0.05
nb_surrogates = 10000

In [60]:
years_1871_2013 = np.arange(1871, 2013+1)

In [61]:
window_length = 70
window_step = 1
time_windows_1871_2013, center_points = sw.get_centered_sliding_windows(window_length, window_step, len(years_1871_2013))

In [62]:
Caesar_1871_2013_det = AMOC_Caesar_det
CWG_1871_2013_det = CWG_log_det

In [63]:
from scipy.ndimage import gaussian_filter1d

def gaussian_smoothing(ts, sigma=2):
    trend = gaussian_filter1d(ts, sigma=sigma)    
    residuals = ts - trend
    return trend, residuals

In [64]:
CWG_trend, CWG_residuals = gaussian_smoothing(CWG_log, sigma=30)

In [65]:
def plot_smoothing(ts, trend, residuals):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=years_1871_2013, y=ts, mode='lines', name='log(CWG)', line=dict(color='#916F8A'), opacity=0.8))
    fig.add_trace(go.Scatter(x=years_1871_2013, y=trend, mode='lines', name='Trend', line=dict(color='#A02C5A')))
    fig.add_trace(go.Scatter(x=years_1871_2013, y=residuals, mode='lines', name='Residuals', line=dict(color='#DE8787'), opacity=0.8))

    fig.update_layout(
        xaxis_title='Year',
        yaxis_title='Value (z-score)',
        #xaxis=dict(tickvals=[0, 2000, 4000, 6000, 8000, 10000], ticktext=['0', '2,000', '4,000', '6,000', '8,000', '10,000'], range=[0, n_points]),
        legend=dict(x=0.03, y=0.95, traceorder='normal', bgcolor='rgba(255, 255, 255, 0.5)'),
        width=500, height=400,
        plot_bgcolor='rgba(215, 238, 244, 0.3)',    # plotting area
        paper_bgcolor='rgba(0, 0, 0, 0)',  # entire plot background
    )

    return fig

In [66]:
fig_CWG_smoothing = plot_smoothing(CWG_log, CWG_trend, CWG_residuals)
fig_CWG_smoothing.show()

In [67]:
# fig_CWG_smoothing.write_image('fig_CWG_smoothing.svg')

In [68]:
# results_Caesar_CWG = compute_CE_for_all_lags(AMOC_Caesar_det, CWG_log_det, graphs=graphsCE, graphAMOC=graphAMOC, time_windows=time_windows_1871_2013, n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, CI=90, detrend=True)
# graphs_Caesar_CWG = create_graph_from_results(results_Caesar_CWG)
# with open('graphs_Caesar_CWG_LinReg_fullDet_tau.pkl', 'wb') as f:
#     pickle.dump(graphs_Caesar_CWG, f)

In [69]:
# compute_and_save_plot_data(graphs_Caesar_CWG, nb_surrogates=nb_surrogates, save_path='boxplot_Caesar_CWG_LinReg_fullDet_tau.pkl')

In [70]:
# with open('boxplot_Caesar_CWG_LinReg_fullDet_tau.pkl', 'rb') as f:
#     boxplot_Caesar_CWG = pickle.load(f)

# fig_boxplot_Caesar_CWG = plot_precomputed_results_boxplot(boxplot_Caesar_CWG)
# fig_boxplot_Caesar_CWG.write_image("fig_boxplot_Caesar_CWG.svg")
# fig_boxplot_Caesar_CWG.show()

In [71]:
# AC_Caesar_1871_2013 = ind.regular_autocorrelation(data=AMOC_Caesar_det, time_windows=time_windows_1871_2013, lag=1, detrend=True)
# AC_CWG_1871_2013 = ind.regular_autocorrelation(data=CWG_1871_2013_det, time_windows=time_windows_1871_2013, lag=1, detrend=True)

In [72]:
# causal_LinReg_Caesar_1871_2013, _, _, bootstrap_LinReg_Caesar_1871_2013, CI_LinReg_Caesar_1871_2013 = pipeline_causalEE_weighted_bootstrap(
#     CausalEffects(graphAMOC, graph_type='stationary_dag', X=[(0, -1)], Y=[(0, 0)], S=None, hidden_variables=None, verbosity=0),
#     AMOC_Caesar_det.reshape(-1, 1), LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows_1871_2013, var_names=[r'$AMOC$'], edge_points_proportion=edge_points_proportion, CI=90, detrend=True)

# data_pair = np.hstack([CWG_1871_2013_det.reshape(-1, 1), Caesar_1871_2013_det.reshape(-1, 1)])
# causal_LinReg_Caesar_CWG_lag4, _, _, bootstrap_LinReg_Caesar_CWG_lag4, CI_LinReg_Caesar_CWG_lag4 = pipeline_causalEE_weighted_bootstrap(
#     CausalEffects(graphsCE[4], graph_type='stationary_dag', X=[(1, -1)], Y=[(1, 0)], S=None, hidden_variables=None, verbosity=0), 
#     data_pair, LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows_1871_2013, var_names=[r'$GrIS$', r'$AMOC$'], edge_points_proportion=edge_points_proportion, CI=90, detrend=True)

In [73]:
# fig_AC_DCE_Caesar_CWG = plot_custom_results(AC_AMOC=AC_Caesar_1871_2013, AC_GIS=AC_CWG_1871_2013,
#                     causal_AMOC=causal_LinReg_Caesar_1871_2013, causal_GIS_AMOC=causal_LinReg_Caesar_CWG_lag4,
#                     CI_AMOC=CI_LinReg_Caesar_1871_2013, CI_GIS_AMOC=CI_LinReg_Caesar_CWG_lag4,
#                     time=years_1871_2013, lag=4, GrIS_proxy='CWG', window_length=window_length, nb_surrogates=nb_surrogates,)
# fig_AC_DCE_Caesar_CWG.write_image("fig_AC_DCE_Caesar_CWG.svg")
# fig_AC_DCE_Caesar_CWG.show()

### Sensitivity analysis

In [74]:
graphAMOC = np.array([[['', '-->', '', '', '', '']],], dtype='<U3')

In [75]:
dataAMOC = np.array(df_1871_2013.AMOC_Caesar)
dataGrIS = np.array(np.log(df_1871_2013.CWG + np.abs(df_1871_2013.CWG.min()) + 1))

In [76]:
window_length_ls = [60, 70, 80]

In [77]:
# sensitivity_Caesar_1871_2013 = sensitivity_analysis(dataAMOC, graphs=[graphAMOC], sigma_ls=[20, 30, 40], window_length_ls=window_length_ls, nb_surrogates=nb_surrogates, detrend=True)

In [78]:
# fig_sensitivity_Caesar_1871_2013 = plot_pvalue_heatmaps_from_results(sensitivity_Caesar_1871_2013, bandwidths=[20, 30, 40], window_lengths=window_length_ls, AMOC_only=True)
# fig_sensitivity_Caesar_1871_2013.show()

In [79]:
# fig_sensitivity_Caesar_1871_2013.write_image("fig_sensitivity_Caesar_1871_2013.svg")

In [80]:
# sensitivity_Caesar_CWG = sensitivity_analysis(dataAMOC, dataGrIS, graphsCE, sigma_ls=[20, 30, 40], window_length_ls=window_length_ls, nb_surrogates=nb_surrogates, detrend=True)

In [81]:
# fig_sensitivity_Caesar_CWG = plot_pvalue_heatmaps_from_results(sensitivity_Caesar_CWG, bandwidths=[20, 30, 40], window_lengths=window_length_ls)
# fig_sensitivity_Caesar_CWG.show()

In [82]:
# fig_sensitivity_Caesar_CWG.write_image("fig_sensitivity_Caesar_CWG.svg")

## Salinity based index and CWG

In [83]:
SNN1_1900_2013 = SNN1[:114]

SNN1_1900_2013_det = SNN1_1900_2013 - gaussian_filter(SNN1_1900_2013, sigma=30)
CWG_1900_2013_det = CWG_log[29:] - gaussian_filter(CWG_log[29:], sigma=30)

In [84]:
years_1900_2013 = np.arange(1900, 2013+1, 1)

In [85]:
window_length = 70
time_windows_1900_2013, _ = sw.get_centered_sliding_windows(window_length=window_length, window_step=1, t_len=len(years_1900_2013))

In [86]:
# AC_SNN1_1900_201 = ind.regular_autocorrelation(data=SNN1_1900_2013_det, time_windows=time_windows_1900_2013, lag=1, detrend=True)
# AC_CWG_1900_2013 = ind.regular_autocorrelation(data=CWG_1900_2013_det, time_windows=time_windows_1900_2013, lag=1, detrend=True)

# causal_LinReg_SNN1_1900_2013, _, _, bootstrap_LinReg_SNN1_1900_2013, CI_LinReg_SNN1_1900_2013 = pipeline_causalEE_weighted_bootstrap(
#     CausalEffects(graphAMOC, graph_type='stationary_dag', X=[(0, -1)], Y=[(0, 0)], S=None, hidden_variables=None, verbosity=0),
#     SNN1_1900_2013_det.reshape(-1, 1), LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows_1900_2013, var_names=[r'$AMOC$'], edge_points_proportion=edge_points_proportion, CI=90, detrend=True)

# data_pair = np.hstack([CWG_1900_2013_det.reshape(-1, 1), SNN1_1900_2013_det.reshape(-1, 1)])
# causal_LinReg_SNN1_CWG_lag4, _, _, bootstrap_LinReg_SNN1_CWG_lag4, CI_LinReg_SNN1_CWG_lag4 = pipeline_causalEE_weighted_bootstrap(
#     CausalEffects(graphsCE[4], graph_type='stationary_dag', X=[(1, -1)], Y=[(1, 0)], S=None, hidden_variables=None, verbosity=0), 
#     data_pair, LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows_1900_2013, var_names=[r'$GrIS$', r'$AMOC$'], edge_points_proportion=edge_points_proportion, CI=90, detrend=True)

In [87]:
# fig_AC_DCE_SNN1_CWG = plot_custom_results(AC_AMOC=AC_SNN1_1900_2013, AC_GIS=AC_CWG_1900_2013,
#                     causal_AMOC=causal_LinReg_SNN1_1900_2013, causal_GIS_AMOC=causal_LinReg_SNN1_CWG_lag4,
#                     CI_AMOC=CI_LinReg_SNN1_1900_2013, CI_GIS_AMOC=CI_LinReg_SNN1_CWG_lag4,
#                     time=years_1900_2013, window_length=window_length, nb_surrogates=nb_surrogates)
# fig_AC_DCE_SNN1_CWG.write_image("fig_AC_DCE_SNN1_CWG.svg")
# fig_AC_DCE_SNN1_CWG.show()

In [88]:
# results_SNN1_CWG = compute_CE_for_all_lags(SNN1_1900_2013_det, CWG_1900_2013_det, graphs=graphsCE, graphAMOC=graphAMOC, time_windows=time_windows_1900_2013, n_bootstrap=n_bootstrap, boot_blocklength=4, CI=90, detrend=True)

# graphs_SNN1_CWG = create_graph_from_results(results_SNN1_CWG)
# with open('graphs_SNN1_CWG_LinReg_fullDet_70_tau.pkl', 'wb') as f:
#     pickle.dump(graphs_SNN1_CWG, f)

In [89]:
# with open('graphs_SNN1_CWG_LinReg_fullDet_55_tau.pkl', 'rb') as f:
#     graphs_SNN1_CWG = pickle.load(f)

In [90]:
# compute_and_save_plot_data(graphs_SNN1_CWG, nb_surrogates=nb_surrogates, save_path='boxplot_SNN1_CWG_LinReg_fullDet_70_tau.pkl')

In [91]:
# with open('boxplot_SNN1_CWG_LinReg_fullDet_70_tau.pkl', 'rb') as f:
#     boxplot_SNN1_CWG = pickle.load(f)

# fig_boxplot_SNN1_CWG = plot_precomputed_results_boxplot(boxplot_SNN1_CWG)
# fig_boxplot_SNN1_CWG.write_image("fig_boxplot_SNN1_CWG.svg")
# fig_boxplot_SNN1_CWG.show()

### Sensitivity analysis

In [92]:
dataAMOC = SNN1_1900_2013
dataGrIS = CWG_log[29:]

In [93]:
window_length_ls = [60, 70, 80]

In [94]:
sensitivity_SNN1_1900_2013 = sensitivity_analysis(dataAMOC, graphs=[graphAMOC], sigma_ls=[20, 30, 40], window_length_ls=window_length_ls, nb_surrogates=nb_surrogates, detrend=True)

Analyzing graph with AMOC only


In [95]:
fig_sensitivity_SNN1_1900_2013 = plot_pvalue_heatmaps_from_results(sensitivity_SNN1_1900_2013, bandwidths=[20, 30, 40], window_lengths=window_length_ls, AMOC_only=True)
fig_sensitivity_SNN1_1900_2013.show()

In [96]:
# fig_sensitivity_SNN1_1900_2013.write_image("fig_sensitivity_SNN1_1900_2013.svg")

In [97]:
# sensitivity_SNN1_CWG = sensitivity_analysis(dataAMOC, dataGrIS, graphsCE, sigma_ls=[20, 30, 40], window_length_ls=window_length_ls, nb_surrogates=nb_surrogates, detrend=True)

In [98]:
# fig_sensitivity_SNN1_CWG = plot_pvalue_heatmaps_from_results(sensitivity_SNN1_CWG, bandwidths=[20, 30, 40], window_lengths=window_length_ls)
# fig_sensitivity_SNN1_CWG.show()

In [99]:
# fig_sensitivity_SNN1_CWG.write_image("fig_sensitivity_SNN1_CWG.svg")

## Caesar index and FWF

In [100]:
FWF_1958_2013_det = np.array((total_freshwater[:56] - gaussian_filter(total_freshwater[:56], sigma=30)).values)
Caesar_1958_2013_det = np.array(df_1871_2013.AMOC_Caesar[87:] - gaussian_filter(df_1871_2013.AMOC_Caesar[87:], sigma=30))

In [101]:
years_1958_2013 = np.arange(1958, 2013+1, 1)

In [102]:
window_length = 30
time_windows_1958_2013, center_points = sw.get_centered_sliding_windows(window_length=window_length, window_step=1, t_len=len(years_1958_2013))

In [103]:
n_bootstrap = 100
boot_blocklength = 3
n_surrogates = 10000

In [104]:
FWF_trend, FWF_residuals = gaussian_smoothing(total_freshwater[:56], sigma=30)

In [105]:
def plot_smoothing(ts, trend, residuals):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=years_1958_2013, y=ts, mode='lines', name='FWF', line=dict(color='#916F8A'), opacity=0.8), secondary_y=False)
    fig.add_trace(go.Scatter(x=years_1958_2013, y=trend, mode='lines', name='Trend', line=dict(color='#A02C5A')), secondary_y=False)
    fig.add_trace(go.Scatter(x=years_1958_2013, y=residuals, mode='lines', name='Residuals', line=dict(color='#DE8787'), opacity=0.8), secondary_y=True)

    fig.update_xaxes(title_text='Year')
    fig.update_yaxes(title_text='Value (km³)', secondary_y=False)
    fig.update_yaxes(title_text='Residuals (km³)', secondary_y=True)

    fig.update_layout(
        legend=dict(x=0.03, y=0.95, traceorder='normal', bgcolor='rgba(255, 255, 255, 0.5)'),
        width=540, height=400,
        plot_bgcolor='rgba(215, 238, 244, 0.3)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
    )

    return fig

In [106]:
# fig_FWF_smoothing = plot_smoothing(total_freshwater[:56], FWF_trend, FWF_residuals)
# fig_FWF_smoothing.show()

In [107]:
# fig_FWF_smoothing.write_image('fig_FWF_smoothing.svg')

In [108]:
# AC_Caesar_1958_2013 = ind.regular_autocorrelation(data=Caesar_1958_2013_det, time_windows=time_windows_1958_2013, lag=1, detrend=True)
# AC_FWF_1958_2013 = ind.regular_autocorrelation(data=FWF_1958_2013_det, time_windows=time_windows_1958_2013, lag=1, detrend=True)

# causal_LinReg_Caesar_1958_2013, _, _, bootstrap_LinReg_Caesar_1958_2013, CI_LinReg_Caesar_1958_2013 = pipeline_causalEE_weighted_bootstrap(
#     CausalEffects(graphAMOC, graph_type='stationary_dag', X=[(0, -1)], Y=[(0, 0)], S=None, hidden_variables=None, verbosity=0),
#     Caesar_1958_2013_det.reshape(-1, 1), LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows_1958_2013, var_names=[r'$AMOC$'], edge_points_proportion=edge_points_proportion, CI=90, detrend=True)

# data_pair = np.hstack([FWF_1958_2013_det.reshape(-1, 1), Caesar_1958_2013_det.reshape(-1, 1)])
# causal_LinReg_Caesar_FWF_lag4, _, _, bootstrap_LinReg_Caesar_FWF_lag4, CI_LinReg_Caesar_FWF_lag4 = pipeline_causalEE_weighted_bootstrap(
#     CausalEffects(graphsCE[4], graph_type='stationary_dag', X=[(1, -1)], Y=[(1, 0)], S=None, hidden_variables=None, verbosity=0), 
#     data_pair, LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows_1958_2013, var_names=[r'$GrIS$', r'$AMOC$'], edge_points_proportion=edge_points_proportion, CI=90, detrend=True)

In [109]:
# fig_AC_DCE_Caesar_FWF = plot_custom_results(AC_AMOC=AC_Caesar_1958_2013, AC_GIS=AC_FWF_1958_2013,
#                     causal_AMOC=causal_LinReg_Caesar_1958_2013, causal_GIS_AMOC=causal_LinReg_Caesar_FWF_lag4,
#                     CI_AMOC=CI_LinReg_Caesar_1958_2013, CI_GIS_AMOC=CI_LinReg_Caesar_FWF_lag4,
#                     time=years_1958_2013, window_length=window_length, nb_surrogates=nb_surrogates)
# fig_AC_DCE_Caesar_FWF.write_image("fig_AC_DCE_Caesar_FWF.svg")
# fig_AC_DCE_Caesar_FWF.show()

In [110]:
#results_Caesar_FWF = compute_CE_for_all_lags(Caesar_1958_2013_det, FWF_1958_2013_det, graphs=graphsCE, graphAMOC=graphAMOC, time_windows=time_windows_1958_2013, n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, CI=90, detrend=True)

# graphs_Caesar_FWF = create_graph_from_results(results_Caesar_FWF)
# with open('graphs_Caesar_FWF_LinReg_fullDet_30_tau.pkl', 'wb') as f:
#     pickle.dump(graphs_Caesar_FWF, f)

In [111]:
#compute_and_save_plot_data(graphs_Caesar_FWF, nb_surrogates=n_surrogates, save_path='boxplot_Caesar_FWF_LinReg_fullDet_30_tau.pkl')

In [112]:
# with open('boxplot_Caesar_FWF_LinReg_fullDet_30_tau.pkl', 'rb') as f:
#     boxplot_Caesar_FWF = pickle.load(f)

# fig_boxplot_Caesar_FWF = plot_precomputed_results_boxplot(boxplot_Caesar_FWF)
# fig_boxplot_Caesar_FWF.write_image("fig_boxplot_Caesar_FWF.svg")
# fig_boxplot_Caesar_FWF.show()

### Sensitivity analysis

In [113]:
dataAMOC = np.array(df_1871_2013.AMOC_Caesar[87:])
dataGrIS = np.array(total_freshwater[:56])

In [114]:
window_length_ls = [20, 30, 40]

In [115]:
# sensitivity_Caesar_1958_2013 = sensitivity_analysis(dataAMOC, graphs=[graphAMOC], sigma_ls=[20, 30, 40], window_length_ls=window_length_ls, nb_surrogates=n_surrogates, detrend=True)

In [116]:
# fig_sensitivity_Caesar_1958_2013 = plot_pvalue_heatmaps_from_results(sensitivity_Caesar_1958_2013, bandwidths=[20, 30, 40], window_lengths=window_length_ls, AMOC_only=True)
# fig_sensitivity_Caesar_1958_2013.show()

In [117]:
# fig_sensitivity_Caesar_1958_2013.write_image("fig_sensitivity_Caesar_1958_2013.svg")

In [118]:
# sensitivity_Caesar_FWF = sensitivity_analysis(dataAMOC, dataGrIS, graphsCE, sigma_ls=[20, 30, 40], window_length_ls=window_length_ls, nb_surrogates=n_surrogates, detrend=True)

In [119]:
# fig_sensitivity_Caesar_FWF = plot_pvalue_heatmaps_from_results(sensitivity_Caesar_FWF, bandwidths=[20, 30, 40], window_lengths=window_length_ls)
# fig_sensitivity_Caesar_FWF.show()

In [120]:
# fig_sensitivity_Caesar_FWF.write_image("fig_sensitivity_Caesar_FWF.svg")

## Salinity based indices and FWF

In [121]:
FWF_1958_2016_det = (total_freshwater - gaussian_filter(total_freshwater, sigma=30)).values
SNN1_1958_2016_det = SNN1[58:117] - gaussian_filter(SNN1[58:117], sigma=30)

In [122]:
years_1958_2016 = np.arange(1958, 2016+1, 1)

In [123]:
window_length = 30
time_windows_1958_2016, center_points = sw.get_centered_sliding_windows(window_length=window_length, window_step=1, t_len=len(years_1958_2016))

In [124]:
n_bootstrap = 100
boot_blocklength = 3

In [125]:
# AC_SNN1_1958_2016 = ind.regular_autocorrelation(data=SNN1_1958_2016_det, time_windows=time_windows_1958_2016, lag=1, detrend=True)
# AC_FWF_1958_2016 = ind.regular_autocorrelation(data=FWF_1958_2016_det, time_windows=time_windows_1958_2016, lag=1, detrend=True)

# causal_LinReg_SNN1_1958_2016, _, _, bootstrap_LinReg_SNN1_1958_2016, CI_LinReg_SNN1_1958_2016 = pipeline_causalEE_weighted_bootstrap(
#     CausalEffects(graphAMOC, graph_type='stationary_dag', X=[(0, -1)], Y=[(0, 0)], S=None, hidden_variables=None, verbosity=0),
#     SNN1_1958_2016_det.reshape(-1, 1), LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows_1958_2016, var_names=[r'$AMOC$'], edge_points_proportion=edge_points_proportion, CI=90, detrend=True)

# data_pair = np.hstack([FWF_1958_2016_det.reshape(-1, 1), SNN1_1958_2016_det.reshape(-1, 1)])
# causal_LinReg_SNN1_FWF_lag4, _, _, bootstrap_LinReg_SNN1_FWF_lag4, CI_LinReg_SNN1_FWF_lag4 = pipeline_causalEE_weighted_bootstrap(
#         CausalEffects(graphsCE[4], graph_type='stationary_dag', X=[(1, -1)], Y=[(1, 0)], S=None, hidden_variables=None, verbosity=0), 
#     data_pair, LinearRegression(), n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, data_transform=StandardScaler(), n_points=n_points, time_windows=time_windows_1958_2016, var_names=[r'$GrIS$', r'$AMOC$'], edge_points_proportion=edge_points_proportion, CI=90, detrend=True)

In [126]:
# fig_AC_DCE_SNN1_FWF = plot_custom_results(AC_AMOC=AC_SNN1_1958_2016, AC_GIS=AC_FWF_1958_2016,
#                     causal_AMOC=causal_LinReg_SNN1_1958_2016, causal_GIS_AMOC=causal_LinReg_SNN1_FWF_lag4,
#                     CI_AMOC=CI_LinReg_SNN1_1958_2016, CI_GIS_AMOC=CI_LinReg_SNN1_FWF_lag4,
#                     time=years_1958_2016, window_length=window_length, nb_surrogates=nb_surrogates)
# fig_AC_DCE_SNN1_FWF.write_image("fig_AC_DCE_SNN1_FWF.svg")
# fig_AC_DCE_SNN1_FWF.show()

In [127]:
# results_SNN1_FWF = compute_CE_for_all_lags(SNN1_1958_2016_det, FWF_1958_2016_det, graphs=graphsCE, graphAMOC=graphAMOC, time_windows=time_windows_1958_2016, n_bootstrap=n_bootstrap, boot_blocklength=boot_blocklength, CI=90, detrend=True)

# graphs_SNN1_FWF = create_graph_from_results(results_SNN1_FWF)
# with open('graphs_SNN1_FWF_LinReg_fullDet_30_tau.pkl', 'wb') as f:
#     pickle.dump(graphs_SNN1_FWF, f)

In [128]:
#compute_and_save_plot_data(graphs_SNN1_FWF, nb_surrogates=n_surrogates, save_path='boxplot_SNN1_FWF_LinReg_fullDet_30_tau.pkl')

In [129]:
# with open('boxplot_SNN1_FWF_LinReg_fullDet_30_tau.pkl', 'rb') as f:
#     boxplot_SNN1_FWF = pickle.load(f)

# fig_boxplot_SNN1_FWF = plot_precomputed_results_boxplot(boxplot_SNN1_FWF)
# fig_boxplot_SNN1_FWF.write_image("fig_boxplot_SNN1_FWF.svg")
# fig_boxplot_SNN1_FWF.show()

### Sensitivity analysis

In [130]:
dataAMOC = np.array(SNN1[58:117])
dataGrIS = np.array(total_freshwater)

In [131]:
window_length_ls = [20, 30, 40]

In [132]:
# sensitivity_SNN1_1959_2016 = sensitivity_analysis(dataAMOC, graphs=[graphAMOC], sigma_ls=[20, 30, 40], window_length_ls=window_length_ls, nb_surrogates=nb_surrogates, detrend=True)

In [133]:
# fig_sensitivity_SNN1_1959_2016 = plot_pvalue_heatmaps_from_results(sensitivity_SNN1_1959_2016, bandwidths=[20, 30, 40], window_lengths=window_length_ls, AMOC_only=True)
# fig_sensitivity_SNN1_1959_2016.show()

In [134]:
# fig_sensitivity_SNN1_1959_2016.write_image("fig_sensitivity_SNN1_1959_2016.svg")

In [135]:
# sensitivity_SNN1_FWF = sensitivity_analysis(dataAMOC, dataGrIS, graphsCE, sigma_ls=[20, 30, 40], window_length_ls=window_length_ls, nb_surrogates=nb_surrogates, detrend=True)

In [136]:
# fig_sensitivity_SNN1_FWF = plot_pvalue_heatmaps_from_results(sensitivity_SNN1_FWF, bandwidths=[20, 30, 40], window_lengths=window_length_ls)
# fig_sensitivity_SNN1_FWF.show()

In [137]:
# fig_sensitivity_SNN1_FWF.write_image("fig_sensitivity_SNN1_FWF.svg")